In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

## TCGA data

In [3]:
TCGA = pd.read_csv('TCGA_data.csv')
TCGA = TCGA.loc[(TCGA.cancer == 'BLCA') | (TCGA.cancer == 'PAAD')| (TCGA.cancer == 'KICH')| (TCGA.cancer == 'KIRC')| (TCGA.cancer == 'KIRP')]
TCGA.loc[(TCGA.cancer == 'BLCA') & (TCGA.y != 'Normal'),'y'] = 'BC'
TCGA.loc[(TCGA.cancer == 'PAAD') & (TCGA.y != 'Normal'),'y'] = 'PC'
TCGA.loc[(TCGA.cancer == 'KICH') & (TCGA.y != 'Normal'),'y'] = 'RC'
TCGA.loc[(TCGA.cancer == 'KIRC') & (TCGA.y != 'Normal'),'y'] = 'RC'
TCGA.loc[(TCGA.cancer == 'KIRP') & (TCGA.y != 'Normal'),'y'] = 'RC'

In [29]:
TCGA.describe()

,cyto.1p36.32,cyto.1p36.31,cyto.1p36.23,cyto.1p36.22,cyto.1p36.21,cyto.1p36.13,cyto.1p36.12,cyto.1p36.11,cyto.1p35.3,cyto.1p35.2,...,cyto.9q31.3,cyto.9q32,cyto.9q33.1,cyto.9q33.2,cyto.9q33.3,cyto.9q34.11,cyto.9q34.12,cyto.9q34.13,cyto.9q34.2,cyto.9q34.3
count,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,...,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000,2877.000000
mean,-0.032079,-0.031753,-0.031455,-0.032120,-0.031903,-0.033995,-0.033825,-0.036324,-0.033226,-0.031052,...,-0.037264,-0.036849,-0.037645,-0.037862,-0.037204,-0.038088,-0.036377,-0.036879,-0.036091,-0.037762
std,0.141242,0.139184,0.138448,0.135821,0.139217,0.134149,0.134824,0.133454,0.132427,0.129602,...,0.156896,0.154853,0.155987,0.158991,0.155327,0.156589,0.157110,0.157137,0.158041,0.168313
min,-1.001000,-0.976300,-0.976300,-0.988600,-0.984200,-0.948200,-0.948200,-0.948200,-0.948200,-0.948200,...,-0.950700,-0.973200,-0.995200,-1.000800,-0.995200,-0.995200,-0.973200,-0.990300,-0.991400,-2.669000
25%,-0.016700,-0.017000,-0.016800,-0.016500,-0.016700,-0.016500,-0.016700,-0.017800,-0.016700,-0.014500,...,-0.028300,-0.027500,-0.027600,-0.027500,-0.027500,-0.027500,-0.026900,-0.026900,-0.025800,-0.026900
50%,0.000700,0.000700,0.000700,0.000700,0.000700,0.000700,0.000700,0.000700,0.000700,0.000800,...,0.000300,0.000300,0.000300,0.000300,0.000300,0.000300,0.000300,0.000300,0.000300,0.000300
75%,0.005200,0.005100,0.005000,0.005000,0.004800,0.004500,0.004300,0.004100,0.004300,0.004400,...,0.004200,0.004200,0.004200,0.004300,0.004300,0.004300,0.004400,0.004500,0.004500,0.004600
max,0.868100,1.083900,0.779600,0.621700,1.362400,0.621700,0.936400,0.604900,0.963900,1.057800,...,0.602400,0.545100,0.545100,0.836500,0.545100,0.545100,1.467700,0.907600,0.782700,0.782700


## urine data

In [7]:
BC_100 = pd.read_csv('BC_32ea_k100_cyto.csv')
Normal_100 = pd.read_csv('Normal_21ea_k100_cyto.csv')
PC_100 = pd.read_csv('PC_20ea_k100_cyto.csv')
RC_100 = pd.read_csv('RC_20ea_k100_cyto.csv')

#drop person id
BC_100.drop(['Unnamed: 0','id'], axis = 1, inplace = True)
Normal_100.drop(['Unnamed: 0','id'], axis = 1, inplace = True)
PC_100.drop(['Unnamed: 0','id'], axis = 1, inplace = True)
RC_100.drop(['Unnamed: 0','id'], axis = 1, inplace = True)

BC_100['cancer'] = 'BC'
Normal_100['cancer'] = 'normal'
PC_100['cancer'] = 'PC'
RC_100['cancer'] = "RC"

K100 = pd.concat([BC_100,Normal_100,PC_100,RC_100], axis = 0)

del [BC_100,Normal_100,PC_100,RC_100]

K100 = K100.dropna(axis=0)
K100.isna().sum().sum()

0

In [28]:
K100.describe()

,cyto.10p11.1,cyto.10p11.21,cyto.10p11.22,cyto.10p11.23,cyto.10p12.1,cyto.10p12.2,cyto.10p12.31,cyto.10p12.32,cyto.10p12.33,cyto.10p13,...,cyto.9q32,cyto.9q33.1,cyto.9q33.2,cyto.9q33.3,cyto.9q34.11,cyto.9q34.12,cyto.9q34.13,cyto.9q34.2,cyto.9q34.3,<NA>
count,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,...,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000
mean,-0.007284,-0.000120,0.020618,-0.002086,0.016443,0.020120,0.017606,0.042091,-0.028045,0.026951,...,-0.077496,-0.043479,-0.047143,-0.017504,-0.008817,0.010125,-0.021967,-0.029835,-0.020170,-0.031824
std,0.158329,0.087372,0.074669,0.082138,0.058898,0.073134,0.079404,0.207909,0.097385,0.083254,...,0.112341,0.129342,0.114587,0.083742,0.087054,0.127445,0.096611,0.096595,0.089733,0.347456
min,-0.518000,-0.233633,-0.175750,-0.222600,-0.221478,-0.250300,-0.282278,-0.528000,-0.285429,-0.260500,...,-0.797393,-0.909913,-0.765303,-0.435227,-0.444300,-0.465200,-0.452895,-0.451857,-0.427387,-1.037000
25%,-0.068833,-0.033783,-0.000402,-0.051500,-0.008505,-0.016663,-0.003326,-0.075250,-0.076036,-0.005390,...,-0.093098,-0.043668,-0.052280,-0.028756,-0.010717,-0.053150,-0.048803,-0.046911,-0.031161,-0.215875
50%,-0.000500,0.004500,0.019679,0.002567,0.012641,0.020225,0.010986,0.044000,-0.036643,0.013940,...,-0.058536,-0.012739,-0.023182,-0.001932,0.008633,0.021200,-0.002684,-0.014071,-0.000097,-0.046250
75%,0.077667,0.026017,0.037777,0.040767,0.036511,0.039813,0.027042,0.147000,0.027143,0.030250,...,-0.029991,0.013679,0.007280,0.022841,0.028567,0.114600,0.023750,0.021911,0.029621,0.168875
max,0.752333,0.552800,0.538179,0.305533,0.319543,0.414600,0.493417,0.735000,0.327857,0.390300,...,0.061821,0.081043,0.052121,0.124841,0.121733,0.230600,0.138421,0.116714,0.101452,0.880500


## 공통 col 추출

In [8]:
a = list(TCGA.columns)
b = list(K100.columns)

new_col = []
for col in a :
    for col2 in b :
        if col==col2 :
            new_col.append(col)       

In [9]:
TCGA_new = TCGA[new_col]
K100_new = K100[new_col]

## 1. scale : 기본 스케일. 평균과 표준편차 사용

In [19]:
X = TCGA_new.drop(['cancer'], axis=1)
X = scale(X)
Y = TCGA['y']
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3)

rf = RandomForestClassifier()
rf.fit(X_train,y_train)

predicted = rf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)

print(f'accuracy score: {accuracy:.3}')

accuracy score: 0.909


In [20]:
X = K100_new.drop(['cancer'], axis=1)
X = scale(X)
Y = K100['cancer']

predicted = rf.predict(X)
accuracy = accuracy_score(Y, predicted)

print(f'accuracy score: {accuracy:.3}')

accuracy score: 0.352


## 2. robust_scale : 중앙값과 IQR사용. 아웃라이어의 영향을 최소화

In [21]:
X = TCGA_new.drop(['cancer'], axis=1)
X = robust_scale(X)
Y = TCGA['y']
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3)

rf = RandomForestClassifier()
rf.fit(X_train,y_train)

predicted = rf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)

print(f'accuracy score: {accuracy:.3}')

accuracy score: 0.911


In [22]:
X = K100_new.drop(['cancer'], axis=1)
X = robust_scale(X)
Y = K100['cancer']

predicted = rf.predict(X)
accuracy = accuracy_score(Y, predicted)

print(f'accuracy score: {accuracy:.3}')

accuracy score: 0.227


## 3. minmax_scale : 최대/최소값이 각각 1, 0이 되도록 

In [23]:
X = TCGA_new.drop(['cancer'], axis=1)
X = minmax_scale(X)
Y = TCGA['y']
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3)

rf = RandomForestClassifier()
rf.fit(X_train,y_train)

predicted = rf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)

print(f'accuracy score: {accuracy:.3}')

accuracy score: 0.91


In [24]:
X = K100_new.drop(['cancer'], axis=1)
X = minmax_scale(X)
Y = K100['cancer']

predicted = rf.predict(X)
accuracy = accuracy_score(Y, predicted)

print(f'accuracy score: {accuracy:.3}')

accuracy score: 0.284


## 4. maxabs_scale :  최대절대값과 0이 각각 1, 0이 되도록 

In [25]:
X = TCGA_new.drop(['cancer'], axis=1)
X = maxabs_scale(X)
Y = TCGA['y']
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3)

rf = RandomForestClassifier()
rf.fit(X_train,y_train)

predicted = rf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)

print(f'accuracy score: {accuracy:.3}')

accuracy score: 0.914


In [26]:
X = K100_new.drop(['cancer'], axis=1)
X = maxabs_scale(X)
Y = K100['cancer']

predicted = rf.predict(X)
accuracy = accuracy_score(Y, predicted)

print(f'accuracy score: {accuracy:.3}')

accuracy score: 0.42
